In [1]:
from os import path
import sys

sys.path.append("../")

import utils

from torch_geometric.utils.convert import from_networkx
import torch
import os
import numpy as np
import torch
import torch.nn.functional as F
from torch_geometric.loader import DataLoader
from sklearn.metrics import f1_score
from gcn_models import GIN, GATC, SAGEe
from bayes_opt import BayesianOptimization

In [2]:
early_stop_thresh = 15
best_macro_f1 = -1

In [3]:
train_dataset = utils.GraphDataset(
    "../data/", "MixedShapesSmallTrain_TRAIN", True, n_quantiles=100
)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

test_dataset = utils.GraphDataset(
    "../data/", "MixedShapesSmallTrain_TEST", True, n_quantiles=100
)
test_loader = DataLoader(test_dataset, batch_size=64)


model = GATC(
    1, [2, 4, 8, 16, 32, 64, 64, 64, 32, 16, 16, 8], pool_method="max", heads=1
)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=5, mode="min"
)

In [4]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data.x.to(torch.float32), data.edge_index, data.batch)
        loss = F.cross_entropy(out, data.y - 1)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader, return_loss=True):
    model.eval()
    y_pred = []
    y_true = []
    loss = 0
    for data in loader:
        y_out = model(data.x.to(torch.float32), data.edge_index, data.batch)
        y_pred.append(y_out.argmax(dim=-1))
        y_true.append(data.y - 1)
        if return_loss:
            loss += float(F.cross_entropy(y_out, data.y - 1)) * data.num_graphs
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    if return_loss:
        return f1_score(y_true=y_true, y_pred=y_pred, average="macro"), loss / len(
            loader.dataset
        )
    return f1_score(y_true=y_true, y_pred=y_pred, average="macro")

In [5]:
for epoch in range(1, 501):
    train()
    train_macro_f1, train_loss = test(train_loader)
    test_macro_f1, test_loss = test(test_loader, True)
    scheduler.step(train_loss)
    print(
        f"Epoch: {epoch:03d}, Train_Loss: {train_loss:02.4f}, Train: {train_macro_f1:01.4f}, Test_Loss: {test_loss:02.4f}, Test: {test_macro_f1:01.4f}"
    )
    if test_macro_f1 > best_macro_f1:
        best_accuracy = test_macro_f1
        best_epoch = epoch
        torch.save(model.state_dict(), "../data/best_model.pth")
    elif epoch - best_epoch > early_stop_thresh:
        print(
            f"Early stopped training at epoch {epoch} best macro F1 {best_macro_f1} in {best_epoch}"
        )
        break

Epoch: 001, Train_Loss: 1.6233, Train: 0.0667, Test_Loss: 1.6172, Test: 0.0850
Epoch: 002, Train_Loss: 1.6189, Train: 0.0667, Test_Loss: 1.6149, Test: 0.0850
Epoch: 003, Train_Loss: 1.6152, Train: 0.0667, Test_Loss: 1.6103, Test: 0.0850
Epoch: 004, Train_Loss: 1.6152, Train: 0.0667, Test_Loss: 1.6113, Test: 0.0850
Epoch: 005, Train_Loss: 1.6135, Train: 0.0667, Test_Loss: 1.6090, Test: 0.0850
Epoch: 006, Train_Loss: 1.6115, Train: 0.0667, Test_Loss: 1.6062, Test: 0.0850
Epoch: 007, Train_Loss: 1.6106, Train: 0.0667, Test_Loss: 1.6040, Test: 0.0850
Epoch: 008, Train_Loss: 1.6109, Train: 0.0667, Test_Loss: 1.6020, Test: 0.0850


KeyboardInterrupt: 

In [6]:
model = SAGE(1, [2, 4, 8, 16, 32, 64, 64, 64, 32, 16, 16, 8], aggr_method="max")
# self,
# in_channels,
# sizes,
# out_channels=5,
# pool_method="max",
# aggr_method="mean",
# normalize=True,

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, patience=5, mode="min"
)

In [7]:
def train():
    model.train()

    total_loss = 0
    for data in train_loader:
        optimizer.zero_grad()
        out = model(data.x.to(torch.float32), data.edge_index, data.batch)
        loss = F.cross_entropy(out, data.y - 1)
        loss.backward()
        optimizer.step()
        total_loss += float(loss) * data.num_graphs
    return total_loss / len(train_loader.dataset)


@torch.no_grad()
def test(loader, return_loss=True):
    model.eval()
    y_pred = []
    y_true = []
    loss = 0
    for data in loader:
        y_out = model(data.x.to(torch.float32), data.edge_index, data.batch)
        y_pred.append(y_out.argmax(dim=-1))
        y_true.append(data.y - 1)
        if return_loss:
            loss += float(F.cross_entropy(y_out, data.y - 1)) * data.num_graphs
    y_pred = np.concatenate(y_pred)
    y_true = np.concatenate(y_true)
    if return_loss:
        return f1_score(y_true=y_true, y_pred=y_pred, average="macro"), loss / len(
            loader.dataset
        )
    return f1_score(y_true=y_true, y_pred=y_pred, average="macro")

In [8]:
for epoch in range(1, 501):
    train()
    train_macro_f1, train_loss = test(train_loader)
    test_macro_f1, test_loss = test(test_loader, True)
    scheduler.step(train_loss)
    print(
        f"Epoch: {epoch:03d}, Train_Loss: {train_loss:02.4f}, Train: {train_macro_f1:01.4f}, Test_Loss: {test_loss:02.4f}, Test: {test_macro_f1:01.4f}"
    )
    if test_macro_f1 > best_macro_f1:
        best_accuracy = test_macro_f1
        best_epoch = epoch
        torch.save(model.state_dict(), "../data/best_model.pth")
    elif epoch - best_epoch > early_stop_thresh:
        print(
            f"Early stopped training at epoch {epoch} best macro F1 {best_macro_f1} in {best_epoch}"
        )
        break

Epoch: 001, Train_Loss: 1.6137, Train: 0.0667, Test_Loss: 1.6037, Test: 0.0588
Epoch: 002, Train_Loss: 1.6112, Train: 0.0667, Test_Loss: 1.6067, Test: 0.0588
Epoch: 003, Train_Loss: 1.6110, Train: 0.0667, Test_Loss: 1.6085, Test: 0.0588
Epoch: 004, Train_Loss: 1.6111, Train: 0.0667, Test_Loss: 1.6118, Test: 0.0588
Epoch: 005, Train_Loss: 1.6114, Train: 0.0667, Test_Loss: 1.6144, Test: 0.0459
Epoch: 006, Train_Loss: 1.6120, Train: 0.0667, Test_Loss: 1.6148, Test: 0.0459
Epoch: 007, Train_Loss: 1.6124, Train: 0.0667, Test_Loss: 1.6133, Test: 0.0459
Epoch: 008, Train_Loss: 1.6128, Train: 0.0667, Test_Loss: 1.6120, Test: 0.0459
Epoch: 009, Train_Loss: 1.6136, Train: 0.0667, Test_Loss: 1.6115, Test: 0.0459
Epoch: 010, Train_Loss: 1.6135, Train: 0.0667, Test_Loss: 1.6114, Test: 0.0459
Epoch: 011, Train_Loss: 1.6135, Train: 0.0667, Test_Loss: 1.6113, Test: 0.0459
Epoch: 012, Train_Loss: 1.6135, Train: 0.0667, Test_Loss: 1.6112, Test: 0.0459
Epoch: 013, Train_Loss: 1.6134, Train: 0.0667, Test_

KeyboardInterrupt: 